In [0]:
from pyspark.sql.functions import col, explode

# 1. Read Bronze Data
raw_df = spark.read.table("healthcare_catalog.raw_data.bronze_healthcare_raw")

# 2. SILVER PATIENTS
silver_patients = (raw_df.select(
                        col("patient.patient_id"),
                        col("patient.name"),
                        col("patient.dob").cast("date").alias("date_of_birth"),
                        col("patient.gender"),
                        col("metadata.event_time").alias("ingested_at")
                    ).dropDuplicates(["patient_id"]))

# 3. BASE ENCOUNTER DATA (Middle step)
enc_base_df = raw_df.withColumn("enc_flat", explode(col("encounters")))

# 4. SILVER ENCOUNTERS
silver_encounters = (enc_base_df.select(
                        col("enc_flat.encounter_id"),
                        col("enc_flat.patient_id"),
                        col("enc_flat.admission_date").cast("date").alias("admission_date"),
                        col("enc_flat.encounter_type"),
                        col("enc_flat.diagnosis")
                    ).dropDuplicates(["encounter_id"]))

# 5. SILVER PROCEDURES 
silver_procedures = (enc_base_df.withColumn("proc_flat", explode(col("enc_flat.procedure")))
                     .select(
                         col("proc_flat.procedure_id"),
                         col("enc_flat.encounter_id").alias("encounter_id"),
                         col("proc_flat.procedure_code"),
                         col("proc_flat.description")
                     ).dropDuplicates(["procedure_id"]))

# 6. Write to Silver Tables with Schema Overwrite
silver_patients.write.mode("overwrite").option("overwriteSchema", "true") \
    .saveAsTable("healthcare_catalog.raw_data.silver_patients")

silver_encounters.write.mode("overwrite").option("overwriteSchema", "true") \
    .saveAsTable("healthcare_catalog.raw_data.silver_encounters")

silver_procedures.write.mode("overwrite").option("overwriteSchema", "true") \
    .saveAsTable("healthcare_catalog.raw_data.silver_procedures")

print("✅ Silver Tables (Patients, Encounters, Procedures) created successfully!")